In [ ]:
from PIL import Image
import os,sys,time,re,json

def get_prompt_png(png_file:str):
    if os.path.isfile(png_file) and png_file.endswith('png'):
        im = Image.open(png_file)
        im.load()  # Needed only for .png EXIF data (see citation above)
        parameters=im.info.get('parameters',None)
        png_info_dict={"filename":png_file,"input_prompt":None,"negative_prompt":None}
        png_para_dict={}
        if parameters:
            input_paras=parameters.split('\n')
            png_info_dict["input_prompt"]=input_paras[0]
            png_info_dict["negative_prompt"]=re.match(r'Negative prompt[:](.*)',input_paras[1]).group(1)
            png_para_dict=text_to_dict(input_paras[-1])
        png_info_dict.update(png_para_dict)
        png_info_dict["postprocessing"]=im.info.get('postprocessing',None)
        png_info_dict["extras"]=im.info.get('extras',None)
        png_info_dict["width"],png_info_dict["height"]=im.width,im.height
        
    return png_info_dict

def text_to_dict(text:str):
    """transform a text to dict data
    text_form=Steps: 20, Sampler: DPM++ 2S a, CFG scale: 7, Seed: 123839547, Face restoration: CodeFormer, Size: 512x768, Model hash: fc2511737a,
    Args:
        text (str): _description_
    """
    sep_text=text.split(', ')
    text_info_dict={}
    for item in sep_text:
        key=item.split(': ')[0]
        vars=item.split(': ')[-1]
        text_info_dict[key]=vars
    return text_info_dict

def comfy_ui_png_info(png_file:str):
    if os.path.isfile(png_file) and png_file.endswith('png'):
        im = Image.open(png_file)
        im.load()  # Needed only for .png EXIF data (see citation above)
        prompt_json=json.loads(im.info.get('prompt',None))
        workflow_json=json.loads(im.info.get('workflow',None))
        # for comfy ui 2x 
        if '8' in prompt_json.keys():
            pos_prompt=prompt_json['8']['inputs']['positive']
            neg_prompt=prompt_json['8']['inputs']['negative']
            model=prompt_json['8']['inputs']['base_ckpt_name']
        elif '4' in prompt_json.keys():
            pos_prompt=prompt_json['4']['inputs']['positive']
            neg_prompt=prompt_json['4']['inputs']['negative']
            model=prompt_json['4']['inputs']['base_ckpt_name']
        return model,pos_prompt,neg_prompt

In [ ]:
filename=r'F:\Coding\PythonProjects\ImageProcessScripts\img\00189-145572448.png'
filename2=r'F:\Coding\PythonProjects\ImageProcessScripts\img\SDXL__00001_.png'
    
model,pos_prompt,neg_prompt=comfy_ui_png_info(filename2)
print(f'model:{model}\npos_prompt:{pos_prompt}\nneg_prompt:{neg_prompt}')